In [12]:
import pandas as pd
import numpy as np

In [13]:
df=pd.read_csv('../data/raw/BreastCancerDS.csv',index_col=0)


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16180 entries, 1 to 19709
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    16180 non-null  string
 1   perimeter_se             13640 non-null  object
 2   radius_worst             13630 non-null  object
 3   concave points_mean      13600 non-null  object
 4   smoothness_mean          13450 non-null  object
 5   area_mean                13425 non-null  object
 6   concavity_se             13475 non-null  object
 7   texture_mean             13565 non-null  object
 8   concavity_worst          13540 non-null  object
 9   smoothness_se            13610 non-null  object
 10  concave points_se        13600 non-null  object
 11  area_worst               13530 non-null  object
 12  compactness_mean         13480 non-null  object
 13  radius_mean              13495 non-null  object
 14  area_se                  13475 non-nul

## Limpiando  el target

In [38]:
df['diagnosis'].value_counts()

B    10245
M     5935
Name: diagnosis, dtype: int64

In [14]:
df=df[(df['diagnosis']=='B')|(df['diagnosis']=='M')]


## Borrando las columnas iuytr y erty

In [26]:
df['erty'].value_counts() ## todos los valores son iguales, se borra erty
df.drop('erty',axis=1,inplace=True)

In [27]:
df[['iuytr','symmetry_mean']] ## estas dos columnas tienen la misma informacion, borrar una
df.drop('iuytr',axis=1,inplace=True)

In [28]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,NaN,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,NaN,0.1724,0.01528,0.01541,NaN
2,307,1144.0,9699.0,0.003472,NaN,246.3,0.003681,14.4,0.01472,0.007389,...,89346,NaN,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,NaN,0.02278,0.09524,409.0,0.01349,18.75,NaN,0.008328,...,85759902,NaN,0.3306,B,0.002386,73.34,NaN,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,NaN,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,NaN,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,NaN,M,NaN,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,NaN,0.01162,0.07561,421.0,0.005949,17.07,0.03046,NaN,...,899187,0.1671,0.2731,B,NaN,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,NaN,585.9,0.007741,15.79,0.03517,NaN,...,90944601,0.1405,0.1859,B,0.002564,NaN,0.1071,NaN,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [17]:
missing = df.isnull().sum()
missing[missing>0]*100/len(df)

perimeter_se               3.275649
radius_worst               3.182942
concave points_mean        3.244747
smoothness_mean            4.295426
area_mean                  4.140915
concavity_se               3.831891
texture_mean               3.522868
concavity_worst            3.213844
smoothness_se              2.626700
concave points_se          3.430161
area_worst                 3.337454
compactness_mean           3.770087
radius_mean                3.491965
area_se                    3.831891
concave points_worst       2.997528
iuytr                      3.831891
fractal_dimension_worst    2.904821
perimeter_worst            3.152040
texture_se                 3.182942
fractal_dimension_mean     3.090235
texture_worst              3.244747
smoothness_worst           3.337454
concavity_mean             4.295426
id                         2.657602
symmetry_mean              3.831891
symmetry_worst             2.843016
fractal_dimension_se       3.059333
perimeter_mean             3

In [18]:
df.dropna()


,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,symmetry_mean,symmetry_worst,diagnosis,erty,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,999765432456788.0,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,211.0,0.2955,B,908765434567,0.001593,999765432456788.0,0.1724,0.01528,0.01541,999765432456788.0
2,307,1144.0,9699.0,0.003472,-88888765432345.0,246.3,0.003681,14.4,0.01472,0.007389,...,-88888765432345.0,0.2991,B,908765434567,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,?,0.02278,0.09524,409.0,0.01349,18.75,?,0.008328,...,?,0.3306,B,908765434567,0.002386,73.34,?,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,0.1671,0.2731,B,908765434567,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,?,...,0.1667,0.3016,B,908765434567,0.002206,73.34,239.0,0.01344,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,?,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,216.0,?,M,908765434567,?,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,rxctf378968 7656463sdfg,0.01162,0.07561,421.0,0.005949,17.07,0.03046,rxctf378968 7656463sdfg,...,0.1671,0.2731,B,908765434567,rxctf378968 7656463sdfg,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,-88888765432345.0,585.9,0.007741,15.79,0.03517,-88888765432345.0,...,0.1405,0.1859,B,908765434567,0.002564,-88888765432345.0,0.1071,-88888765432345.0,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,0.2275,0.3651,M,908765434567,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [19]:
table = pd.pivot_table(df, index =["index","id"])

In [20]:
table


erty
index id                                   
0     842302                   908765434567
      rxctf378968 7656463sdfg  908765434567
1     842517                   908765434567
      842517.0                 908765434567
2     84300903                 908765434567
...                                     ...
655   913505.0                 908765434567
      999765432456788          908765434567
656   913512                   908765434567
      913512.0                 908765434567
      ?                        908765434567

[1490 rows x 1 columns]

In [22]:
df['index']=df['index'].astype('string')
list= ['index', 'perimeter_se', 'radius_worst', 'concave points_mean',
       'smoothness_mean', 'area_mean', 'concavity_se', 'texture_mean',
       'concavity_worst', 'smoothness_se', 'concave points_se', 'area_worst',
       'compactness_mean', 'radius_mean', 'area_se', 'concave points_worst',
       'iuytr', 'fractal_dimension_worst', 'perimeter_worst', 'texture_se',
       'fractal_dimension_mean', 'texture_worst', 'smoothness_worst',
       'concavity_mean', 'id', 'symmetry_mean', 'symmetry_worst', 'diagnosis',
       'fractal_dimension_se', 'perimeter_mean', 'compactness_worst',
       'symmetry_se', 'compactness_se', 'radius_se']
for i in list:
    df[i]= df[i].str.strip()

In [23]:
list=['rxctf378968 7656463sdfg','-88888765432345.0','999765432456788.0','?']
for i in list:
       df.replace(i,np.nan,inplace=True)


In [24]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,symmetry_mean,symmetry_worst,diagnosis,erty,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,NaN,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,211.0,0.2955,B,908765434567,0.001593,NaN,0.1724,0.01528,0.01541,NaN
2,307,1144.0,9699.0,0.003472,NaN,246.3,0.003681,14.4,0.01472,0.007389,...,NaN,0.2991,B,908765434567,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,NaN,0.02278,0.09524,409.0,0.01349,18.75,NaN,0.008328,...,NaN,0.3306,B,908765434567,0.002386,73.34,NaN,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,0.1671,0.2731,B,908765434567,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,NaN,...,0.1667,0.3016,B,908765434567,0.002206,73.34,239.0,0.01344,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,NaN,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,216.0,NaN,M,908765434567,NaN,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,NaN,0.01162,0.07561,421.0,0.005949,17.07,0.03046,NaN,...,0.1671,0.2731,B,908765434567,NaN,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,NaN,585.9,0.007741,15.79,0.03517,NaN,...,0.1405,0.1859,B,908765434567,0.002564,NaN,0.1071,NaN,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,0.2275,0.3651,M,908765434567,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16180 entries, 0 to 16179
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    16180 non-null  object
 1   perimeter_se             13640 non-null  object
 2   radius_worst             13630 non-null  object
 3   concave points_mean      13600 non-null  object
 4   smoothness_mean          13450 non-null  object
 5   area_mean                13425 non-null  object
 6   concavity_se             13475 non-null  object
 7   texture_mean             13565 non-null  object
 8   concavity_worst          13540 non-null  object
 9   smoothness_se            13610 non-null  object
 10  concave points_se        13600 non-null  object
 11  area_worst               13530 non-null  object
 12  compactness_mean         13480 non-null  object
 13  radius_mean              13495 non-null  object
 14  area_se                  13475 non-nul

In [98]:
df=df.astype('float')

In [99]:
from sklearn import preprocessing
label_encoding = preprocessing.LabelEncoder()
df['diagnosis'] = label_encoding.fit_transform(df['diagnosis'])

## 0 es beningno y 1 es maligno


In [105]:
df['diagnosis']=df['diagnosis'].astype('category')

## 0 es beningno y 1 es maligno

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16180 entries, 0 to 16179
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   index                    16180 non-null  float64 
 1   perimeter_se             13640 non-null  float64 
 2   radius_worst             13630 non-null  float64 
 3   concave points_mean      13600 non-null  float64 
 4   smoothness_mean          13450 non-null  float64 
 5   area_mean                13425 non-null  float64 
 6   concavity_se             13475 non-null  float64 
 7   texture_mean             13565 non-null  float64 
 8   concavity_worst          13540 non-null  float64 
 9   smoothness_se            13610 non-null  float64 
 10  concave points_se        13600 non-null  float64 
 11  area_worst               13530 non-null  float64 
 12  compactness_mean         13480 non-null  float64 
 13  radius_mean              13495 non-null  float64 
 14  area_s

In [90]:
df.describe()

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
count,16180.00000,13640.000000,13630.000000,13600.000000,13450.000000,13425.000000,13475.000000,13565.000000,13540.000000,13610.000000,...,1.472000e+04,13485.000000,13600.000000,16180.000000,13555.000000,13450.000000,13725.000000,13580.000000,13570.000000,13595.000000
mean,328.05068,2536.409364,323.328936,2.424616,3.835104,656.435978,1.147903,19.416823,24.441271,0.007034,...,3.075898e+13,16.305977,31.056838,0.366811,0.010388,92.300212,25.303201,0.219976,0.145002,81.108863
std,189.47624,1738.237381,1680.786074,16.207594,19.928257,349.181590,17.733254,4.385227,106.068910,0.003130,...,1.824612e+14,52.516520,92.196651,0.481949,0.199383,24.473417,95.437704,2.066789,1.474177,292.899986
min,0.00000,0.771400,7.930000,0.000000,0.052630,143.500000,0.000000,9.710000,0.000000,0.001713,...,-8.888877e+13,0.116700,0.156500,0.000000,0.000895,43.790000,0.027290,0.007882,0.002252,0.111500
25%,165.75000,1482.000000,13.290000,0.020270,0.086410,420.500000,0.015140,16.330000,0.121100,0.005033,...,8.654680e+05,0.163400,0.252300,0.000000,0.002217,75.490000,0.150800,0.015020,0.013400,0.235100
50%,326.00000,2143.000000,15.300000,0.033700,0.095920,556.700000,0.026260,18.900000,0.257100,0.006307,...,9.079150e+05,0.181300,0.288400,0.000000,0.003053,86.910000,0.229700,0.018525,0.020480,0.343800
75%,494.00000,3168.000000,19.920000,0.077520,0.106100,782.700000,0.042560,21.870000,0.426725,0.008109,...,8.912944e+06,0.202700,0.331300,1.000000,0.004463,104.300000,0.385600,0.023240,0.032470,0.590700
max,656.00000,9807.000000,9981.000000,162.000000,123.000000,2501.000000,396.000000,39.280000,1252.000000,0.031130,...,9.997654e+14,304.000000,544.000000,1.000000,6.000000,188.500000,1058.000000,31.000000,27.000000,2873.000000


In [107]:
df.to_parquet("../data/interim/BreastCancer.parquet", index = False)


In [108]:
df=pd.read_parquet("../data/interim/BreastCancer.parquet", engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16180 entries, 0 to 16179
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    16180 non-null  float64
 1   perimeter_se             13640 non-null  float64
 2   radius_worst             13630 non-null  float64
 3   concave points_mean      13600 non-null  float64
 4   smoothness_mean          13450 non-null  float64
 5   area_mean                13425 non-null  float64
 6   concavity_se             13475 non-null  float64
 7   texture_mean             13565 non-null  float64
 8   concavity_worst          13540 non-null  float64
 9   smoothness_se            13610 non-null  float64
 10  concave points_se        13600 non-null  float64
 11  area_worst               13530 non-null  float64
 12  compactness_mean         13480 non-null  float64
 13  radius_mean              13495 non-null  float64
 14  area_se               